# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing].
Please download the data and place it in your data_path at "./data."

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

In [3]:
data_path = 'data/'

In [4]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data)
y_test_tensor = torch.LongTensor(test_label)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_data.npy'

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [7]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Show Data

# Build simple Deep learning model

In [32]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, 512), # Input dimention is 64 channel * 795 time point, and use 256 units for first NN layer
            nn.ReLU(), # Use ReLu function for NN training
            nn.Linear(512, 256), # 256 NN units to 128 units
            nn.ReLU(),
            nn.Linear(256, 128),#  128 NN units to 64 units
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(32, num_classes), # num_classes is 5 (hello,” “help me,” “stop,” “thank you,” and “yes”)
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for multi-class classification
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)

        # import pdb;pdb.set_trace()
        x = self.classifier(x)
        return x

In [44]:
num_classes = 5 # setting final output class
model = EEGAutoencoderClassifier(num_classes).to(device)
criterion = nn.NLLLoss() # Use NLLLoss function to optimize

In [50]:
optimizer = optim.Adam(model.parameters(), lr=0.0001) # Setting parameters learning rate = 0.0001

In [51]:
num_epochs = 20 # setting training epochs (Number of training iterations)
for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/20, Loss: 0.07186178863048553
Epoch 2/20, Loss: 0.02947368659079075
Epoch 3/20, Loss: 0.005113187246024609
Epoch 4/20, Loss: 0.002775899600237608
Epoch 5/20, Loss: 0.0029923273250460625
Epoch 6/20, Loss: 0.003529709530994296
Epoch 7/20, Loss: 0.006247045937925577
Epoch 8/20, Loss: 0.03191513195633888
Epoch 9/20, Loss: 0.04916848614811897
Epoch 10/20, Loss: 0.015277862548828125
Epoch 11/20, Loss: 0.01034935936331749
Epoch 12/20, Loss: 0.0064894356764853
Epoch 13/20, Loss: 0.0038196661043912172
Epoch 14/20, Loss: 0.0014319031033664942
Epoch 15/20, Loss: 0.0010980068473145366
Epoch 16/20, Loss: 0.028833908960223198
Epoch 17/20, Loss: 0.052052371203899384
Epoch 18/20, Loss: 0.03473959118127823
Epoch 19/20, Loss: 0.2223990559577942
Epoch 20/20, Loss: 0.21791142225265503


In [49]:
model.eval() # Evaluate your model
correct = 0
total = 0
inputs = []
outcomes = []
with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        outcomes+=predicted
        inputs += labels

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')
#print(outcomes)

Test Accuracy: 51.14%


In [37]:
def output_interpret(out):
    i = out.item()
    if i == 0:
        return "hello"
    if i == 1:
        return "help me"
    if i == 2:
        return "stop"
    if i == 3:
        return "thank you"
    if i == 4:
        return "yes"
                
    return "fail"

In [38]:
def data_and_predictions():
    pred = list(map(output_interpret,outcomes))
    data = list(map(output_interpret,inputs))
    result = []
    
    for i in range(0,len(pred)):
        result = result + [(data[i] , pred[i])]
    
    return result

print(data_and_predictions())

[('thank you', 'help me'), ('yes', 'hello'), ('help me', 'hello'), ('stop', 'yes'), ('help me', 'yes'), ('yes', 'stop'), ('thank you', 'thank you'), ('help me', 'thank you'), ('stop', 'stop'), ('stop', 'hello'), ('stop', 'thank you'), ('hello', 'yes'), ('stop', 'yes'), ('help me', 'help me'), ('help me', 'hello'), ('stop', 'yes'), ('help me', 'help me'), ('yes', 'yes'), ('yes', 'yes'), ('help me', 'yes'), ('hello', 'hello'), ('help me', 'thank you'), ('hello', 'help me'), ('stop', 'yes'), ('thank you', 'hello'), ('hello', 'hello'), ('thank you', 'thank you'), ('thank you', 'yes'), ('thank you', 'stop'), ('thank you', 'stop'), ('hello', 'hello'), ('thank you', 'thank you'), ('yes', 'yes'), ('stop', 'stop'), ('yes', 'thank you'), ('hello', 'hello'), ('help me', 'stop'), ('yes', 'yes'), ('yes', 'yes'), ('hello', 'stop'), ('yes', 'stop'), ('help me', 'hello'), ('hello', 'yes'), ('stop', 'stop'), ('yes', 'yes'), ('thank you', 'thank you'), ('hello', 'yes'), ('hello', 'yes'), ('stop', 'stop'

In [2]:
(pd.DataFrame.from_dict(data_and_predictions(), orient='index')).to_csv('model_outcomes.csv', header=False)

NameError: name 'data_and_predictions' is not defined